第十一組：<br>
111356001 資管碩一 陳靖汝<br>
111356004 資管碩一 唐芸飛<br>
108306037 資管大四 施瑋翔<br>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
import random

import re
from collections import Counter

### 模型建立的資料集介紹
* 需要建立模型的檔案為 shopee_item_category.csv
* 檔案資料大小為 253285 rows × 3 columns
* 3 個 columns 分別為 "商品名稱"、"大類"、"小類"

**清理目標：**去除商品名稱、數量、英文、與商品名稱中那些和商品本質無關的編號和標籤（如：免運、單位）去除

In [ ]:
# import shopee data
df = pd.read_csv('/content/gdrive/Shareddrives/文字探勘/期中考/shopee_item_category.csv')
df

,商品名稱,大類,小類
0,PHILIPS 飛利浦 全自動義式咖啡機-EP3246 (金)+湛盧咖啡豆券8張(24包),家用電器,咖啡機與周邊
1,PHILIPS 飛利浦 小旋風電動洗鞋機 (GCA1000),家用電器,其他家電
2,PHILIPS 飛利浦 2021旗艦款八合一乾濕兩用拔刮美體刀 除毛刀 BRE740,美妝保養,除毛器材
3,PHILIPS 飛利浦 電鬍刀刀頭 RQ11,美妝保養,電動刮鬍刀
4,PHILIPS 飛利浦 全自動義式咖啡機-EP3246 (金),家用電器,咖啡機與周邊
...,...,...,...
253280,白底青平安扣 贈精美中國繩乙入 天然緬甸硬玉A貨【文華珠寶翡翠專賣店】,愛好與收藏品,原石水晶
253281,Q版可愛牛牛玉墜 天然緬甸硬玉A貨 贈中國繩乙入【文華珠寶翡翠專賣店】,愛好與收藏品,原石水晶
253282,冰種平安扣 起螢放光 購買即贈中國繩乙入 天然緬甸硬玉A貨【文華珠寶翡翠專賣店】,愛好與收藏品,原石水晶
253283,冰種觀音玉墜 贈精美中國繩乙條 天然緬甸硬玉A貨【文華珠寶翡翠專賣店】,愛好與收藏品,原石水晶


In [ ]:
# 確認各個 column 不重複的值共有多少筆
col_name = df.columns
for col in col_name:
  print("表格中，不重複之\"" + col + "\"共有" + str(len(df[col].unique())) + "筆")

表格中，不重複之"商品名稱"共有253072筆
表格中，不重複之"大類"共有27筆
表格中，不重複之"小類"共有632筆


### Sample 策略
方法：sample 各個小類 50 筆<br>
目的：
1. 確保每個小類別都被抽樣到
2. 避免隨機抽樣的偏誤

In [ ]:
# 確認小類別最少的筆數
min_small_count = min(Counter(df['小類']).values())
print(min_small_count)

50


In [ ]:
# sample 各個小類 50 筆資料
sample_df = pd.DataFrame()
for i in range(len(df['小類'].unique())):
    result_df = df[df['小類'] == df['小類'].unique()[i]]
    sample_df = sample_df.append(result_df.sample(n = 50, random_state=1)) #radom_state 確保資料抽取一致
sample_df

,商品名稱,大類,小類
144211,SIROCA 自動研磨咖啡機 SC-A3510(K) 【全國電子】,家用電器,咖啡機與周邊
27434,日本NICOH 電動冷熱奶泡機 現貨 廠商直送,家用電器,咖啡機與周邊
27882,Panasonic 國際牌- 四人份全自動雙研磨美式咖啡機 NC-R601 廠商直送,家用電器,咖啡機與周邊
235987,日本Siroca 新一代 自動研磨咖啡機-紅 A1210,家用電器,咖啡機與周邊
64275,超值合購 荷蘭公主 全自動研磨美式咖啡機+冰熱奶泡機 249406+243000,家用電器,咖啡機與周邊
...,...,...,...
253237,大件春帶財如意玉墜 贈精美中國繩乙件 天然緬甸硬玉A貨【文華珠寶翡翠專賣店】,時尚配件,珠寶 / 玉石
146030,【飾品控SPANCONNY】神祈・薰衣草紫水晶 白月光 天然水晶手鍊 手鏈 手串 增智慧 招...,時尚配件,珠寶 / 玉石
253246,天然緬甸硬玉A貨 開運招財貔貅玉墜 【文華珠寶翡翠專賣店】,時尚配件,珠寶 / 玉石
118467,City Diamond引雅 天然小米粒珍珠 時尚眼鏡鍊 眼鏡鏈【手作設計系列】,時尚配件,珠寶 / 玉石


In [ ]:
product_name = sample_df['商品名稱'].values
product_name

array(['SIROCA 自動研磨咖啡機  SC-A3510(K) 【全國電子】', '日本NICOH 電動冷熱奶泡機 現貨 廠商直送',
       'Panasonic 國際牌- 四人份全自動雙研磨美式咖啡機 NC-R601 廠商直送', ...,
       '天然緬甸硬玉A貨 開運招財貔貅玉墜 【文華珠寶翡翠專賣店】',
       'City Diamond引雅 天然小米粒珍珠 時尚眼鏡鍊 眼鏡鏈【手作設計系列】',
       '點睛品 六字大明咒 瑪瑙轉運珠黃金吊墜(附贈紅繩)'], dtype=object)

#### 去除【】,［］, [], （）, (), 《》, 〔〕, 「」內容

In [ ]:
for i in range(len(product_name)):
  
  # print('before', product_name[i])

  # 去除【】,［］, [], （）, (), 《》, 〔〕, 「」內容
  removed = re.sub(u"\【.*?\】", '', product_name[i])
  removed = re.sub(u"\［.*?\］", '', removed)
  removed = re.sub(u"\[.*?\]", '', removed)
  removed = re.sub(u"\（.*?\）", '', removed)
  removed = re.sub(u"\(.*?\)", '', removed)
  removed = re.sub(u"\《.*?\》", '', removed)
  removed = re.sub(u"\〔.*?\〕", '', removed)
  removed = re.sub(u"\「.*?\」", '', removed)

  product_name[i] = removed
  # print('after ', product_name[i])

  # print('-' * 40)

#### 停用詞移除
Stop words：
1. 空格
2. ｜, |, │, ︱, （, (, ), 【, /, ／,&,＆,.,+,＋,#,＃,$,＄（其中包含每個符號的全型以及半型）<br>
3. 與產品無相關的詞：入、組、現貨、折
4. 國家名稱：加拿大、英國、西班牙、澳洲、韓國、法國、瑞士、日本

策略：將 stop words 移除

In [ ]:
stop_word = [' ','｜','|','│','︱','（','(',')','【','/','／','&','＆','.','+','＋','#','＃','$','＄','支','入','組','現貨','折','加拿大', '英國', '西班牙', '澳洲', '韓國', '法國', '瑞士', '日本']
for i in range(len(product_name)):

  # print('before', product_name[i])

  for j in stop_word:
    if j in product_name[i]:
      removed_stop_word = product_name[i].replace(j, '')
      product_name[i] = removed_stop_word
  
  # print('after ', product_name[i])

  # print('-' * 40)

#### 處理非中文的資料
策略：
1. 移除大部分英文以及數字
2. 判別有 T 的是否為 吸排t、t恤、素t、短袖上衣、寬鬆短袖、大學 T

In [ ]:
# 判斷字串是否為中文
def is_all_chinese(strs):
  for _char in strs:
    if not '\u4e00' <=_char <= '\u9fa5':
      return False
  return True

In [ ]:
pl = []

# 將t恤的t字與tee改為「踢」
for product in product_name:
  product = product.lower()
  
  # print('before', product)
  
  for i, char in enumerate(product):
    # t前後是中文 eg."素t短袖上衣寬鬆短袖"
    if char == 't' and 0 < i < (len(product) - 1):
        if (is_all_chinese(product[i-1]) and is_all_chinese(product[i+1])):
            p = list(product)
            p[i] = '踢'
            product = ''.join(p)
    # t在最前面 t在最後面 eg."t恤大學t"
    elif char == 't' and ((i == 0 and is_all_chinese(product[i+1])) or (i == (len(product) - 1) and is_all_chinese(product[i-1]))):
        p = list(product)
        p[i] = '踢'
        product = ''.join(p)
    # "tee"
    elif char == 't' and 0 <= i <= (len(product) - 3) and product[i+1] == 'e' and product[i+2] == 'e':
      p = list(product)
      p[i] = '踢'
      product = ''.join(p)

    if is_all_chinese(product[i]) == False:
        p = list(product)
        p[i] = ' '
        product = ''.join(p)

  # print('after ', product)
  pl.append(product)
  
  # print('-' * 40)

In [ ]:
# 移除剩餘的空格
stop_word = [' ']
for i in range(len(pl)):
  # print('before', pl[i])

  for j in stop_word:
    if j in pl[i]:
      removed_stop_word = pl[i].replace(j, '')
      pl[i] = removed_stop_word
  
  # print('after ', pl[i])

  # print('-' * 40)

In [ ]:
sample_df['product_name'] = pl
sample_df

,商品名稱,大類,小類,product_name
144211,SIROCA自動研磨咖啡機SC-A3510,家用電器,咖啡機與周邊,自動研磨咖啡機
27434,NICOH電動冷熱奶泡機廠商直送,家用電器,咖啡機與周邊,電動冷熱奶泡機廠商直送
27882,Panasonic國際牌-四人份全自動雙研磨美式咖啡機NC-R601廠商直送,家用電器,咖啡機與周邊,國際牌四人份全自動雙研磨美式咖啡機廠商直送
235987,Siroca新一代自動研磨咖啡機-紅A1210,家用電器,咖啡機與周邊,新一代自動研磨咖啡機紅
64275,超值合購荷蘭公主全自動研磨美式咖啡機冰熱奶泡機249406243000,家用電器,咖啡機與周邊,超值合購荷蘭公主全自動研磨美式咖啡機冰熱奶泡機
...,...,...,...,...
253237,大件春帶財如意玉墜贈精美中國繩乙件天然緬甸硬玉A貨,時尚配件,珠寶 / 玉石,大件春帶財如意玉墜贈精美中國繩乙件天然緬甸硬玉貨
146030,神祈・薰衣草紫水晶白月光天然水晶手鍊手鏈手串增智慧招貴人穩定情緒,時尚配件,珠寶 / 玉石,神祈薰衣草紫水晶白月光天然水晶手鍊手鏈手串增智慧招貴人穩定情緒
253246,天然緬甸硬玉A貨開運招財貔貅玉墜,時尚配件,珠寶 / 玉石,天然緬甸硬玉貨開運招財貔貅玉墜
118467,CityDiamond引雅天然小米粒珍珠時尚眼鏡鍊眼鏡鏈,時尚配件,珠寶 / 玉石,引雅天然小米粒珍珠時尚眼鏡鍊眼鏡鏈


# jieba斷詞

In [ ]:
product_name = sample_df['product_name'].values

In [ ]:
# !pip install jieba
import jieba

In [ ]:
# jieba斷詞
# 搜索引擎模式
jiebaResults = []
for m in product_name:
  seg_list = jieba.lcut_for_search(m)
  jiebaResults.append(seg_list)

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.034 seconds.
DEBUG:jieba:Loading model cost 1.034 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [ ]:
# 將斷詞結果利用空格分開
jieba_strings = []
for i in range(len(jiebaResults)):
  jieba_string = ''
  for v in range(len(jiebaResults[i])):
    jieba_string = jieba_string + jiebaResults[i][v]
    if v < (len(jiebaResults[i]) - 1):
      jieba_string = jieba_string + ' '
  jieba_strings.append(jieba_string)

In [ ]:
sample_df['jieba_results'] = jiebaResults
sample_df['jieba_string'] = jieba_strings
sample_df

,商品名稱,大類,小類,product_name,jieba_results,jieba_string
144211,SIROCA自動研磨咖啡機SC-A3510,家用電器,咖啡機與周邊,自動研磨咖啡機,"[自動, 研磨, 咖啡, 機]",自動 研磨 咖啡 機
27434,NICOH電動冷熱奶泡機廠商直送,家用電器,咖啡機與周邊,電動冷熱奶泡機廠商直送,"[電動, 冷熱奶, 泡, 機廠, 商直, 送]",電動 冷熱奶 泡 機廠 商直 送
27882,Panasonic國際牌-四人份全自動雙研磨美式咖啡機NC-R601廠商直送,家用電器,咖啡機與周邊,國際牌四人份全自動雙研磨美式咖啡機廠商直送,"[國際牌, 人份, 四人份, 全自動雙, 研磨, 美式, 咖啡, 機廠, 商直, 送]",國際牌 人份 四人份 全自動雙 研磨 美式 咖啡 機廠 商直 送
235987,Siroca新一代自動研磨咖啡機-紅A1210,家用電器,咖啡機與周邊,新一代自動研磨咖啡機紅,"[一代, 新一代, 自動, 研磨, 咖啡, 機紅]",一代 新一代 自動 研磨 咖啡 機紅
64275,超值合購荷蘭公主全自動研磨美式咖啡機冰熱奶泡機249406243000,家用電器,咖啡機與周邊,超值合購荷蘭公主全自動研磨美式咖啡機冰熱奶泡機,"[超值, 合購, 荷蘭, 公主, 全自動, 研磨, 美式, 咖啡, 機冰, 熱奶, 泡機]",超值 合購 荷蘭 公主 全自動 研磨 美式 咖啡 機冰 熱奶 泡機
...,...,...,...,...,...,...
253237,大件春帶財如意玉墜贈精美中國繩乙件天然緬甸硬玉A貨,時尚配件,珠寶 / 玉石,大件春帶財如意玉墜贈精美中國繩乙件天然緬甸硬玉貨,"[大件, 春帶財, 如意, 玉墜, 贈, 精美, 中國, 繩乙件, 天然, 緬甸, 硬玉, 貨]",大件 春帶財 如意 玉墜 贈 精美 中國 繩乙件 天然 緬甸 硬玉 貨
146030,神祈・薰衣草紫水晶白月光天然水晶手鍊手鏈手串增智慧招貴人穩定情緒,時尚配件,珠寶 / 玉石,神祈薰衣草紫水晶白月光天然水晶手鍊手鏈手串增智慧招貴人穩定情緒,"[神祈, 衣草, 薰衣草, 水晶, 紫水晶, 月光, 白月光, 天然, 水晶, 手, 鍊, ...",神祈 衣草 薰衣草 水晶 紫水晶 月光 白月光 天然 水晶 手 鍊 手鏈 手串 增 智慧 招...
253246,天然緬甸硬玉A貨開運招財貔貅玉墜,時尚配件,珠寶 / 玉石,天然緬甸硬玉貨開運招財貔貅玉墜,"[天然, 緬甸, 硬玉, 貨開運, 招財, 貔貅, 玉墜]",天然 緬甸 硬玉 貨開運 招財 貔貅 玉墜
118467,CityDiamond引雅天然小米粒珍珠時尚眼鏡鍊眼鏡鏈,時尚配件,珠寶 / 玉石,引雅天然小米粒珍珠時尚眼鏡鍊眼鏡鏈,"[引雅, 天然, 小, 米粒, 珍珠, 時尚, 眼鏡, 鍊, 眼鏡, 鏈]",引雅 天然 小 米粒 珍珠 時尚 眼鏡 鍊 眼鏡 鏈


In [ ]:
# 匯出資料
sample_df.to_csv('shopee_jieba.csv', encoding='utf_8_sig')